<a href="https://colab.research.google.com/github/dijkstra001/ia-bot-trader/blob/main/IA_bot_trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IA Bot Trader**

Esse projeto tem como objetivo desenvolver uma IA para realizar recomendações de compra e venda de criptomoedas.

Os dados utilizados são da Exchange Bitmex e a API utilizada está disponível no link: https://www.bitmex.com/app/apiOverview

***Obs: Ao utilizar API para requisição dos dados, cuidado para não exceder o número de requisições.***

In [1]:
!pip install -q ccxt

In [2]:
!pip install -q bayesian-optimization==1.2

In [3]:
import csv
import ccxt
import time
import random
import types
import pkg_resources
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pprint import pprint
from bayes_opt import BayesianOptimization
from google.colab import drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def generate_csv(archive, data):
  with open(archive, mode = 'w') as archive_csv:
    archive_csv.write("Date,Open,High,Low,Close,Adj Close,Volume\n")

    csv_writer = csv.writer(archive_csv, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    csv_writer.writerows(data)

In [6]:
def exchange_connection(exchange, max_retries, symbol, timeframe, since, limit):
  num_retries = 0

  try:
    num_retries += 1
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since)
    return ohlcv

  except Exception:
      if num_retries > max_retries:
        raise

In [7]:
def extract_data(exchange, max_retries, symbol, timeframe, since, limit):
  earliest_timestamp = exchange.milliseconds()

  timeframe_duration_in_sec = exchange.parse_timeframe(timeframe)
  timeframe_duration_in_ms = timeframe_duration_in_sec * 1000
  timedelta = limit * timeframe_duration_in_ms

  all_ohlcv = []

  while True:
    fetch_since = earliest_timestamp - timedelta
    ohlcv = exchange_connection(exchange, max_retries, symbol, timeframe, fetch_since, limit)

    if ohlcv[0][0] >= earliest_timestamp:
      break
    earliest_timestamp = ohlcv[0][0]
    all_ohlcv = ohlcv + all_ohlcv
    print(f'{len(all_ohlcv)} registros extraídos de {exchange.iso8601(all_ohlcv[0][0])} a {exchange.iso8601(all_ohlcv[-1][0])}')

    if fetch_since < since:
      break
  
  return all_ohlcv

In [8]:
def extract_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    exchange = getattr(ccxt, exchange_id)({'enableRateLimit': True,})
    
    if isinstance(since, str):
        since = exchange.parse8601(since)
    exchange.load_markets()
    
    ohlcv = extract_data(exchange, max_retries, symbol, timeframe, since, limit)
    key = 0
    
    for item in ohlcv:
        epoch = int(item[0]) / 1000
        ohlcv[key][0] = datetime.utcfromtimestamp(epoch).strftime('%Y-%m-%d')
        ohlcv[key][5] = int(item[5])
        ohlcv[key].append(ohlcv[key][5])
        ohlcv[key][5] = ohlcv[key][4]
        key += 1
    ohlen = len(ohlcv)
    pprint("Número de Registros: " + str(ohlen))
    
    """if ohlen > 399:
        ohrem = ohlen - 399
        pprint("Removendo: " + str(ohrem))
        ohlcv = ohlcv[ohrem:]""" # Se não tiver recursos suficientes (ram e memória) descomente esse código
    
    generate_csv(filename, ohlcv)
    print('Salvos', len(ohlcv), 'registros no arquivo', filename)

In [9]:
# Exchange: https://www.bitmex.com/app/apiOverview
exchange = "bitmex"
symbol = "BTC/USD"
timeframe = "1d"
since = "2018-01-01T00:00:00Z"
max_retries = 3
outfile = "/content/drive/My Drive/ia-bot-trader/dataset.csv" # Aponte para o diretório onde você deseja salvar os dados

In [10]:
extract_to_csv(outfile, exchange, max_retries, symbol, timeframe, since, 100)

100 registros extraídos de 2021-02-04T00:00:00.000Z a 2021-05-14T00:00:00.000Z
200 registros extraídos de 2020-10-27T00:00:00.000Z a 2021-05-14T00:00:00.000Z
300 registros extraídos de 2020-07-19T00:00:00.000Z a 2021-05-14T00:00:00.000Z
400 registros extraídos de 2020-04-10T00:00:00.000Z a 2021-05-14T00:00:00.000Z
500 registros extraídos de 2020-01-01T00:00:00.000Z a 2021-05-14T00:00:00.000Z
600 registros extraídos de 2019-09-23T00:00:00.000Z a 2021-05-14T00:00:00.000Z
700 registros extraídos de 2019-06-15T00:00:00.000Z a 2021-05-14T00:00:00.000Z
800 registros extraídos de 2019-03-07T00:00:00.000Z a 2021-05-14T00:00:00.000Z
900 registros extraídos de 2018-11-27T00:00:00.000Z a 2021-05-14T00:00:00.000Z
1000 registros extraídos de 2018-08-19T00:00:00.000Z a 2021-05-14T00:00:00.000Z
1100 registros extraídos de 2018-05-11T00:00:00.000Z a 2021-05-14T00:00:00.000Z
1200 registros extraídos de 2018-01-31T00:00:00.000Z a 2021-05-14T00:00:00.000Z
1300 registros extraídos de 2017-10-23T00:00:00.0

# **Análise Exploratória:**

In [11]:
dataset = pd.read_csv(outfile)
dataset.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-10-23,5986.0,6095.2,5638.7,5853.9,5853.9,1013064819
1,2017-10-24,5853.9,5853.9,5454.0,5517.4,5517.4,950744064
2,2017-10-25,5517.4,5758.9,5366.0,5732.0,5732.0,710909859
3,2017-10-26,5732.0,6008.0,5669.6,5888.4,5888.4,773138211
4,2017-10-27,5888.4,6013.0,5678.0,5769.3,5769.3,632848491


In [12]:
dataset.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1300.000000,1300.000000,1300.000000,1300.000000,1300.000000,1.300000e+03
mean,12907.312077,13322.246923,12449.785462,12941.044769,12941.044769,2.314642e+09
std,13065.850111,13506.975124,12557.535260,13104.518059,13104.518059,1.422983e+09
min,3183.000000,3226.000000,3121.000000,3183.000000,3183.000000,6.654364e+07
25%,6685.500000,6828.250000,6486.375000,6688.125000,6688.125000,1.313947e+09
50%,8812.000000,9030.000000,8551.250000,8817.250000,8817.250000,2.064845e+09
75%,11210.000000,11548.625000,10840.125000,11212.500000,11212.500000,2.912033e+09
max,63674.000000,65000.000000,62131.000000,63674.000000,63674.000000,1.362211e+10


In [13]:
# Dados de fechamento:
close = dataset.Close.values.tolist()

# Parâmetros para versão base do modelo:
window_size = 30
skip = 5
l = len(close) - 1

## **Estratégia de Treinamento: Deep Evolution Strategy (OpenIA):**

In [14]:
class PoliticsTrader:
  inputs = None

  def __init__(self, weights, reward_function, population_size, sigma, learning_rate):
        
        self.weights = weights
        self.reward_function = reward_function
        self.population_size = population_size
        self.sigma = sigma
        self.learning_rate = learning_rate

  def get_weights_population(self, weights, population):
    
        weights_population = []
        
        for index, i in enumerate(population):
            jittered = self.sigma * i
            weights_population.append(weights[index] + jittered)
        
        return weights_population

  def get_weights(self):
        return self.weights

  def train(self, epoch = 100, print_every = 1):
        
        lasttime = time.time()
        
        for i in range(epoch):
            
            population = []
            rewards = np.zeros(self.population_size)

            for k in range(self.population_size):
                x = []
                for w in self.weights:
                    x.append(np.random.randn(*w.shape))
                population.append(x)
            
            for k in range(self.population_size):
                weights_population = self.get_weights_population(self.weights, population[k])
                rewards[k] = self.reward_function(weights_population)
            rewards = (rewards - np.mean(rewards)) / np.std(rewards)
            
            for index, w in enumerate(self.weights):
                A = np.array([p[index] for p in population])
                self.weights[index] = (w + self.learning_rate / (self.population_size * self.sigma) * np.dot(A.T, rewards).T)
            
            if (i + 1) % print_every == 0:
                print('Iteração %d. Recompensa: %f' % (i + 1, self.reward_function(self.weights)))
        
        print('Tempo Total de Treinamento:', time.time() - lasttime, 'segundos')


## **Arquitetura do Modelo de Rede Neural:**

In [15]:
class Model:
    
    def __init__(self, input_size, layer_size, output_size):
        
        self.weights = [np.random.randn(input_size, layer_size),
                        np.random.randn(layer_size, output_size),
                        np.random.randn(layer_size, 1),
                        np.random.randn(1, layer_size),]

    def predict(self, inputs):
        feed = np.dot(inputs, self.weights[0]) + self.weights[-1]
        decision = np.dot(feed, self.weights[1])
        buy = np.dot(feed, self.weights[2])
        return decision, buy

    def get_weights(self):
        return self.weights

    def set_weights(self, weights):
        self.weights = weights

## **Configuração do BOT:**

In [16]:
def get_state(data, t, n):
    d = t - n + 1
    block = data[d : t + 1] if d >= 0 else -d * [data[0]] + data[0 : t + 1]
    states = []
    for i in range(n - 1):
        states.append(block[i + 1] - block[i])
    return np.array([states])

In [17]:
class Trader:

    def __init__(self, population_size, sigma, learning_rate, model, money, max_buy, max_sell, skip, window_size,):
        
        self.window_size = window_size
        self.skip = skip
        self.POPULATION_SIZE = population_size
        self.SIGMA = sigma
        self.LEARNING_RATE = learning_rate
        self.model = model
        self.initial_money = money
        self.max_buy = max_buy
        self.max_sell = max_sell
        self.es = PoliticsTrader(self.model.get_weights(),
                                 self.get_reward,
                                 self.POPULATION_SIZE,
                                 self.SIGMA,
                                 self.LEARNING_RATE,)

    def execute(self, sequence):
        decision, buy = self.model.predict(np.array(sequence))
        return np.argmax(decision[0]), int(buy[0])

    def get_reward(self, weights):
        
        initial_money = self.initial_money
        starting_money = initial_money
        
        self.model.weights = weights

        state = get_state(close, 0, self.window_size + 1)

        inventory = []
        quantity = 0
        
        for t in range(0, l, self.skip):
            
            action, buy = self.execute(state)
            
            next_state = get_state(close, t + 1, self.window_size + 1)
          
            if action == 1 and initial_money >= close[t]:
                if buy < 0:
                    buy = 1
                if buy > self.max_buy:
                    buy_units = self.max_buy
                else:
                    buy_units = buy
                    
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
            
            elif action == 2 and len(inventory) > 0:
                if quantity > self.max_sell:
                    sell_units = self.max_sell
                else:
                    sell_units = quantity
                    
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell

            state = next_state
        return ((initial_money - starting_money) / starting_money) * 100

    def fit(self, iterations, checkpoint):
        self.es.treinamento(iterations, print_every = checkpoint)

    def make_investiment(self):
        
        initial_money = self.initial_money
        starting_money = initial_money
        
        state = get_state(close, 0, self.window_size + 1)
              
        states_sell = []
        states_buy = []
        inventory = []
        quantity = 0
        
        for t in range(0, l, self.skip):
            
            action, buy = self.execute(state)
            next_state = get_state(close, t + 1, self.window_size + 1)
            
            if action == 1 and initial_money >= close[t]:
                if buy < 0:
                    buy = 1
                if buy > self.max_buy:
                    buy_units = self.max_buy
                else:
                    buy_units = buy
                
                total_buy = buy_units * close[t]
                initial_money -= total_buy
                inventory.append(total_buy)
                quantity += buy_units
                states_buy.append(t)
                
                print('Dia %d: comprar %d unidades ao preço de %f, saldo total %f' % (t, buy_units, total_buy, initial_money))
            
            elif action == 2 and len(inventory) > 0:
                bought_price = inventory.pop(0)
                if quantity > self.max_sell:
                    sell_units = self.max_sell
                else:
                    sell_units = quantity
                if sell_units < 1:
                    continue
                    
                quantity -= sell_units
                total_sell = sell_units * close[t]
                initial_money += total_sell
                states_sell.append(t)
                
                try:
                    invest = ((total_sell - bought_price) / bought_price) * 100
                except:
                    invest = 0
                
                print('Dia %d, vender %d unidades ao preço de %f, investimento %f %%, saldo total %f,' % (t, sell_units, total_sell, invest, initial_money))
            
            # Próximo estado
            state = next_state

        # Investimento
        invest = ((initial_money - starting_money) / starting_money) * 100
        
        print('\nGanho Total %f, Valor Total Investido %f' % (initial_money - starting_money, invest))
        
        plt.figure(figsize = (20, 10))
        plt.plot(close, label = 'Valor Real de Fechamento', c = 'g')
        plt.plot(close, 'X', label = 'Previsão de Compra', markevery = states_buy, c = 'b')
        plt.plot(close, 'o', label = 'Previsão de Venda', markevery = states_sell, c = 'r')
        plt.legend()
        plt.show()

## **Encontrando o melhor Trader:**

In [18]:
def best_trader(window_size, skip, population_size, sigma, learning_rate, size_network):
    
    model = Model(window_size, size_network, 3)
    trader = Trader(population_size, sigma, learning_rate, model, 10000, 5, 5, skip, window_size,)
    
    try:
        trader.fit(100, 1000)
        return trader.es.reward_function(trader.es.weights)
    except:
        return 0

def find_best_trader(window_size, skip, population_size, sigma, learning_rate, size_network):
    
    global accbest
    
    param = {'window_size': int(np.around(window_size)),
             'skip': int(np.around(skip)),
             'population_size': int(np.around(population_size)),
             'sigma': max(min(sigma, 1), 0.0001),
             'learning_rate': max(min(learning_rate, 0.5), 0.000001),
             'size_network': int(np.around(size_network)),}
    
    print('\nBuscando Parâmetros %s' % (param))
    
    investment = best_trader(**param)
    print('Após 100 iterações o investimento foi de %f' % (investment))
    return investment

## **Otimização Bayesiana Para os Hiperparâmetros do Modelo:**